In [1]:
#importer vos librairies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import datetime as DT
import time
import re
import pickle
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from stop_words import get_stop_words
from joblib import dump, load

In [6]:
class DataHandler:
    def __init__(self) :
        self.dataprice = None
        self.datalisting = None
        self.merge = None
    def get_data(self):
        print("_ _ Fetch Data From bucket _ _")
        self.dataprice=pd.read_csv("DataCSV/price_availability.csv",sep=';')  # What is csv : comma separateur virgule
        self.datalisting=pd.read_csv("DataCSV/listings_final.csv",sep=';')
        return "_ _  data loaded _ _\nFiles : \n - listing {} \n - prices {} ".format(self.datalisting.shape,self.dataprice.shape)
    def group_data(self):
        print(" - - - Start Data Merging - - - ")
        data = self.dataprice.groupby('listing_id')['local_price'].mean()
        self.merge = pd.merge(data, self.datalisting, on='listing_id')
        print(" - - - data merged - - - ")
    def get_process_data(self):
        print(" - - - Start Data Processing - - - ")
        self.get_data()
        self.group_data()
        print(self.merge)
        return self.merge
        print("-- End of DataHandling --") 

In [7]:
d = DataHandler()

In [8]:
d.get_process_data()
%time

 - - - Start Data Processing - - - 
_ _ Fetch Data From bucket _ _
 - - - Start Data Merging - - - 
 - - - data merged - - - 
     listing_id  local_price  Unnamed: 0  \
0         56093   170.000000          12   
1         57207    49.952756          13   
2        114543   107.374026          19   
3        149534   169.000000           9   
4        164255    75.876209          28   
..          ...          ...         ...   
994    28684174   725.175781         662   
995    28709644   475.000000         745   
996    28751412   117.000000          88   
997    28774896   156.397468         159   
998    28792796    49.184211         998   

                                                  name          type   city  \
0                          Beau duplex dans le Marais    entire_home  Paris   
1                             Belle Chambre pour court  private_room  Paris   
2                    Charming 1bdr 55m² - Eiffel Tower   entire_home  Paris   
3                   GREAT WAR

In [9]:
d.merge.dtypes

listing_id                    int64
local_price                 float64
Unnamed: 0                    int64
name                         object
type                         object
city                         object
neighborhood                 object
latitude                    float64
longitude                   float64
person_capacity               int64
beds                          int64
bedrooms                      int64
bathrooms                   float64
is_rebookable                  bool
is_new_listing                 bool
is_fully_refundable            bool
is_host_highly_rated           bool
is_business_travel_ready       bool
pricing_weekly_factor       float64
pricing_monthly_factor      float64
dtype: object

### Methodes :
##### separe des types
##### methode delete duplicated
##### enléve les NA
##### drop useless futures columns
##### deux types de quantitifs : catégories et les valeursa 
##### on pourra avoir les prix en focntion des arrondissements
##### methode appelle toutes

In [10]:
class FeatureRecipe:

    def __init__(self,data:pd.DataFrame):
        """ TODO : __init__ """

        self.data = data
        self.continuous = None # float
        self.categorical = None # Object
        self.discrete = None # int
        self.other = None
        self.datetime = None

    def separate_variable_types(self) -> None:
        """" TODO : Diviser les types de variables dans un tableau """
        print("-- Diviser les types de variables dans un tableau --")
        self.continuous = [] # float
        self.categorical = [] # Object
        self.discrete = [] # int   
        self.other = [] # other ( bool )
        for col in self.data.columns :
            if self.data[col].dtypes == int :
                self.discrete.append(self.data[col])
            elif self.data[col].dtypes == object :
                self.categorical.append(self.data[col])
            elif self.data[col].dtypes == float :
                self.continuous.append(self.data[col])
            else:
                self.other.append(self.data[col])
                          
        print(" The Amount of sepereted types:\n Categories {} \n Discrete {} \n Continuous {} \n Other {}".format(len(self.categorical),len(self.discrete),len(self.continuous),len(self.other)))
    
    def drop_uselessf(self):
        """ TODO : Supprimer les colonnes inutiles du dataset """
        print("-- Dropping uselese Columns--")

        if "Unnamed: 0" in self.data.columns :
            self.data.drop(['Unnamed: 0'],axis='columns',inplace=True)

        for col in self.data.columns :
            if self.data[col].isna().sum() == len(self.data[col]) :
                self.data.drop([col],axis='columns',inplace=True)

        print("--Done Dropping--")
    
    def get_duplicates(self):
        duplicates = []
        for col in range(self.data.shape[1]):
            for scol in range(col+1,self.data.shape[1]):
                if self.data.iloc[:,col].equals(self.data.iloc[:,scol]):
                    duplicates.append(self.data.iloc[:,scol].name)
        return duplicates
    
    def drop_duplicate(self):
        # comparer les colonnes et voir si elles sont dupliquées
        print("dropping duplicated rows")
        self.data.drop_duplicates(inplace=True)
        duplicates = self.get_duplicates()
        for col in duplicates:
            print("dropping column :{}".format(col))
            self.data.drop([col], axis='columns', inplace=True)
        print("duplicated rows dropped")

    def drop_nanp(self, threshold: float):
        """ 
            TODO : Supprimer un pourcentage de NA du dataset 

            Threshold is inserted from 0 to 100 float variable
        """
        def deal_nanp(data:pd.DataFrame, threshold: float):
            bf=[]
            for c in self.data.columns.to_list():
                if self.data[c].isna().sum()/self.data.shape[0] >= threshold:
                    bf.append(c)
            print("{} feature have more than {} NaN ".format(len(bf), threshold))
            print('\n\n - - - features - - -  \n {}'.format(bf))
            return bf 
        
        self.data = self.data.drop(deal_nanp(self.data, threshold), axis=1)
        print('Some NaN features droped according to {} thresold'.format(threshold))
    
    def deal_dtime(self):
        """ TODO : Traiter les DateTime """
        pass

    def prepare_data(self, threshold: float):
        print(" - - - Start Preparing Data - - - ")
        self.drop_uselessf()
        self.drop_duplicate()
        self.drop_nanp(threshold)
        self.separate_variable_types()
        self.deal_dtime()   
        print("-- End of FeatureRecipe --") 

In [11]:
fr = FeatureRecipe(d.merge)

In [12]:
fr.prepare_data(0.4)

 - - - Start Preparing Data - - - 
-- Dropping uselese Columns--
--Done Dropping--
dropping duplicated rows
dropping column :is_business_travel_ready
duplicated rows dropped
0 feature have more than 0.4 NaN 


 - - - features - - -  
 []
Some NaN features droped according to 0.4 thresold
-- Diviser les types de variables dans un tableau --
 The Amount of sepereted types:
 Categories 4 
 Discrete 4 
 Continuous 6 
 Other 4
-- End of FeatureRecipe --


In [13]:
d.merge

,listing_id,local_price,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,pricing_weekly_factor,pricing_monthly_factor
0,56093,170.000000,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,48.867284,2.358431,4,2,1,1.0,False,False,True,True,0.88,1.0
1,57207,49.952756,Belle Chambre pour court,private_room,Paris,Vaugirard,48.846184,2.304455,2,1,1,1.0,False,False,True,False,0.87,1.0
2,114543,107.374026,Charming 1bdr 55m² - Eiffel Tower,entire_home,Paris,NaN,48.849530,2.290219,2,1,1,1.0,False,False,True,True,0.90,0.9
3,149534,169.000000,GREAT WARM FULL APT LE HAUT MARAIS,entire_home,Paris,NaN,48.866360,2.361844,4,2,1,1.0,False,False,True,True,1.00,0.4
4,164255,75.876209,Perfect place in Le Marais - Paris,entire_home,Paris,3e arrondissement,48.861398,2.364299,4,2,1,1.0,False,False,True,False,1.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,28684174,725.175781,Chambre familiale vue jardin avec petit-déjeun...,private_room,Paris,Ternes,48.879223,2.292382,5,0,1,1.0,False,True,True,False,1.00,1.0
995,28709644,475.000000,LORD BYRON-SPACE& STYLE IN 8TH EME,entire_home,Paris,Champs-Elysées,48.872202,2.298349,4,2,1,1.0,False,True,True,False,1.00,1.0
996,28751412,117.000000,Malesherbes Monceau Monsen,entire_home,Paris,Monceau,48.880923,2.314568,2,1,0,1.0,False,True,True,False,1.00,1.0
997,28774896,156.397468,5 min to invalides and 10 min to eiffel tower,entire_home,Paris,Invalides - Ecole Militaire,48.852915,2.314519,2,1,1,1.0,False,True,True,False,1.00,1.0


In [10]:
class FeatureExtractor:
    """
    Feature Extractor class
    """    
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.df = data
        self.flist = flist
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    def extract(self):
        print("-- X and y Values to be put while discarding the flist that we dont need--")
        for col in self.flist:
            if col in self.df:
                self.df.drop(col,axis="columns",inplace=True)

        print("-- Extraction Done --")

    def split(self,size:float,rand:int,y:str):
        print("-- Using Split mehtode --")
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.df.loc[:,self.df.columns!=y], self.df[y], test_size=size, random_state=rand)
        print("-- Afficher la shape de vos données --")

    def get_process_data(self,size:float,rand:int,y:str):
        self.extract()
        self.split(size,rand,y)
        return self.X_train, self.X_test, self.y_train, self.y_test
        print("-- Done processing Feature Extractor --")



  print("self.X_train.shape = {}, self.y_train.shape = {},\n\nself.X_test.shape = {}, self.y_test.shape = {}".format(self.X_train.shape, self.y_train.shape,self.X_test.shape, self.y_test.shape))
        
        print("-- créer l'objet de régression et entrainer le sur notre ensemble d'entraînement --")
        regr= LinearRegression()
        model= regr.fit(self.X_train,self.y_train)

        print("-- On affiche le vecteur des coefficients pour interpréter rapidement le modèle --")
        
        print("Model Coefficients so basicaly the X columns values for y values = {} , \n Model Intercept Value = {} , \n Score Result = {}".format(model.coef_ , model.intercept_, regr.score(self.X_test,self.y_test)))

        print("-- Test Train Split Done --")


In [11]:
flist = ['listing_id','city','neighborhood','latitude','longitude','is_rebookable','is_new_listing','is_fully_refundable','is_host_highly_rated','is_business_travel_ready','pricing_weekly_factor','pricing_monthly_factor','name','type']      


In [12]:
fe = FeatureExtractor(d.merge,flist)
X_train,X_test,y_train,y_test = fe.get_process_data(0.3,42,'local_price')

-- X and y Values to be put while discarding the flist that we dont need--
-- Extraction Done --
-- Using Split mehtode --
-- Afficher la shape de vos données --


In [13]:
X_train

,person_capacity,beds,bedrooms,bathrooms
728,4,2,1,1.0
630,2,1,1,1.0
394,6,4,3,1.5
777,2,2,0,1.0
598,2,1,1,1.0
...,...,...,...,...
106,1,1,1,1.0
270,6,3,2,1.0
860,4,2,2,1.0
435,5,1,1,1.0


In [14]:
y_train

728    298.366755
630    109.421875
394    330.000000
777     40.067010
598     70.434109
          ...    
106     40.234987
270    382.621359
860    115.000000
435    104.789474
102     92.281330
Name: local_price, Length: 699, dtype: float64

In [ ]:
class ModelBuilder:
    """
        Class for train and print results of ml model 
    """
    def __init__(self, model_path: str = None, save: bool = False):
        self.model_path = model_path
        self.save = save
        self.reg = LinearRegression()
        #self.time = DT.datetime.now()

    def __repr__(self): # class courier python , affichage par default isntancié , methode __str__ 
        return f'Model Builder : model_path = {self.model_path} , save = {self.save}.'

    def train(self,X,y):
        print(" - - - Training Start - - - ")
        self.reg.fit(X,y)
        print(" - - - Finish training - - - ")

    def predict_test(self, X) -> np.ndarray:  # certain ligne of X_test or y_test
        print(" - - - Tesitng Certain Ligne - - - ")
        self.reg.predict(X[0])        
        print(" - - - Finish Certain Ligne - - - ")

    def predict_from_dump(self, X) -> np.ndarray: #
        print(" - - - Tesitng From Dump - - - ")
        self.reg.predict(X)
        print(" - - - Finished Testing From Dump - - - ")

    def save_model(self, model_path:str): # joblib saving de predict_test of fit , et on faire le dump de joblib par la focntion de predict_from_dump
        #with the format : 'model_{}_{}'.format(date)
        print(" - - - Saving Model - - - ")
        joblib.dump(self.reg,"model_base.joblib")
        print(" - - - Finished Saving Model - - - ")

    def print_accuracy(self): # accuracy c'est le score 
        print(" - - - Accuracy Printing - - - ")
        acc=self.reg.score(X_test,y_test)
        print(acc)
        print(" - - - Finished Accuracy - - - ")

    def load_model(self): # à partir de fichier joblib , je le mets en instance 
        #self.model en plsu de model baf... si j'ai pas chargé alors pas de modéle,s'il y a alors charge
        try:
            #load model
            return joblib.load("{}/model_base.joblib".format(self.model_path))
            ldm = joblib.load("{}/model_base.joblib".format(self.model_path))
            print("File Loaded Successfully")
        except:
            print("File doesn't exist.You must save the model first")

    


In [ ]:
m=ModelBuilder()

In [ ]:
m = ModelBuilder() 
m.train(X_train, y_train)
m.print_accuracy()
m.save_model(path)